# Homework 1

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from matplotlib_inline import backend_inline
backend_inline.set_matplotlib_formats('svg')

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
# !mkdir data
# !wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-01.parquet -O ../data/fhv_tripdata_2021-01.parquet
# !wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-02.parquet -O ../data/fhv_tripdata_2021-02.parquet

## Q1

In [3]:
df1 = pd.read_parquet('../data/fhv_tripdata_2021-01.parquet')
df2 = pd.read_parquet('../data/fhv_tripdata_2021-02.parquet')

for df in [df1, df2]:
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60


In [4]:
print("Q1:", df1.shape[0])

Q1: 1154112


## Q2

In [5]:
print("Q2:", df1.duration.mean())

Q2: 19.167224093791006


## Q3

In [6]:
df1.isna().mean()

dispatching_base_num      0.000000
pickup_datetime           0.000000
dropOff_datetime          0.000000
PUlocationID              0.830307
DOlocationID              0.140558
SR_Flag                   1.000000
Affiliated_base_number    0.000767
duration                  0.000000
dtype: float64

## Q4

In [7]:
a = len(set(df1.PUlocationID.unique()) | set(df2.PUlocationID.unique()))
b = len(set(df1.DOlocationID.unique()) | set(df2.DOlocationID.unique()))
print("Q4:", a + b - 2) # remove nans

Q4: 526


## Q5 and Q6

In [8]:
df1 = df1[(df1.duration >= 1) & (df1.duration <= 60)]
df2 = df2[(df2.duration >= 1) & (df2.duration <= 60)]


categorical = ['PUlocationID', 'DOlocationID']

# df1 = df1.dropna(subset=categorical)
# df2 = df2.dropna(subset=categorical)
df1 = df1.fillna(-1)
df2 = df2.fillna(-1)


# For dict vectorizer: int = ignored, str = one-hot
df1[categorical] = df1[categorical].astype(int).astype(str)
df2[categorical] = df2[categorical].astype(int).astype(str)

train_dicts = df1[categorical].to_dict(orient='records')
valid_dicts = df2[categorical].to_dict(orient='records')

dv = DictVectorizer()
dv.fit(train_dicts)

X_train = dv.transform(train_dicts)
y_train = df1.duration.values

X_valid = dv.transform(valid_dicts)
y_valid = df2.duration.values

X_train.shape, X_valid.shape

((1109826, 525), (990113, 525))

In [9]:
lr = LinearRegression()
lr.fit(X=X_train, y=y_train)

print("Q5:", mean_squared_error(y_train, lr.predict(X_train), squared=False))
print("Q6:", mean_squared_error(y_valid, lr.predict(X_valid), squared=False))

Q5: 10.528519107210428
Q6: 11.014283211658533
